<a href="https://colab.research.google.com/github/financieras/dataframe/blob/main/2024/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 3
Ahora nos toca un poco de estadística.

A Winston Churchill (¿quién era Winston Churchill?), se le atribuye la sentencia que él solo se fía de las estadísticas que el mismo ha trucado. Nosotros desde luego no hemos falsificado nada, pero la verdad es que siempre hay muchas formas de crear estadísticas, y por esto es importante echar un vistazo a los datos que utilizamos, y pensar si el promedio es el método adecuado, o si preferimos aplicar la mediana.

En realidad, nos hemos saltado unos pasos importantes en la exploración de los datos. Siempre hay que comprobar si los datos son completos (en nuestro caso lo son) y si hay valores atípicos que cambian los resultados, sobre todo cuando trabajamos con el promedio.

Os doy un ejemplo:

Queremos saber cuál es el país más rico, el país A o el país B. Lo primero que hacemos es fijarnos en el PIB (¿qué es el PIB?). Entonces, el país B es el más rico. No obstante, si tomamos en cuenta que el país B es mucho más grande, esto cambia. Si dividimos el PiB por el número de habitantes, resulta que el país A sale mucho mejor parado. Esto significa que los habitantes del país A son más ricos?

Depende: en el país A hay una persona muy rica, y el resto es bastante pobre. Esto hace que casi todo el mundo dispone de mucho menos que la renta media, es decir el valor atípico (la persona rica) afecta mucho el promedio. Entonces aplicamos la mediana en vez del promedio, y vemos que el valor mediano en el país A es mucho más bajo que en el país B, mientras el promedio es más alto en A que en B.

### Ejemplo

| Indicador   | País A        | País B        |
|-------------|--------------|---------------|
| PIB         | 1000         | 2000          |
| Habitantes  | 10           | 40            |
| Media       | 100/hab.     | 50/hab.       |
| H1          | 700          | 200           |
| H2          | 100          | 100           |
| H3-H8       | 30           | 50            |
| H9-H10      | 10           | -             |
| H31-H40     | -            | 10            |
| Mediana     | 30           | 50            |


#### Notación de la tabla
La notación H1, H2, ..., H40 representa magnitudes asociadas a la distribución de la renta, donde cada H indica un grupo o individuo dentro una muestra ordenada por ingresos (por ejemplo, H1 sería el que más renta tiene, H40 el de menor renta en una muestra de 40 habitantes). Esto se utiliza en tablas que resumen cómo está repartido el PIB entre los distintos habitantes o grupos del país, indicando cuánto recibe cada uno.

- H1: Puede representar el individuo o grupo con mayor ingreso o renta dentro del país, según la distribución ordenada del PIB per cápita.
- H40: Sería el individuo o grupo de menor renta si se tienen 40 habitantes y se han ordenado de mayor a menor según ingresos.

Esta notación suele usarse cuando se analiza desigualdad económica, por ejemplo mostrando la concentración de riqueza en los primeros (H1, H2) frente al resto (H3-H40), y permite calcular indicadores como la mediana, la media y visualizar la distribución de la renta de manera sencilla.

Resulta que con el promedio gana el país A porque hay un habitante con una renta muy alta, y con la mediana el  país B, porque en el cálculo de la mediana el habitante superrico del país A ya no tiene tanto peso. Al final y al cabo, tenemos que decidir si utilizamos el PIB, el promedio de la renta per cápita, o el valor mediano de renta per cápita, para sacar nuestras conclusiones.

## Introducción a la Task 3.1
> A ver qué pasa si utilizamos la mediana en vez del promedio.

**Añadir a la tabla de la Task 2.2. una columna con la mediana de candidatos, y crea el scatterplot de task 2.3 basado en el valor mediano de candidatos.**

Ahora, nos toca investigar si en el caso de los candidatos queremos aplicar el promedio (tal y como lo hemos hecho) o la mediana. Es interesante ver que, en algunas tecnologías, el valor promedio y mediano varía bastante. Juega con la nueva tabla, ordenándola según el promedio, y la mediana, y compara el scatterplot con el promedio con el scatterplot con la mediana.

# Preguntas Task 3.1.
¿Cuáles de las siguientes tecnologías (Git, Java, Python y Windows) mejora su posición con respecto al resto si se aplica la mediana en vez del promedio?

Esta es la pregunta tal y como aparece en el test:

¿Cuáles de las siguientes tecnologías mejoran su posición con respecto al resto si se aplica la mediana en vez del promedio?
Select one or more:

- ☐ Linux
- ☐ Python
- ☐ SQL
- ☐ Java

In [8]:
!git clone https://github.com/financieras/dataframe.git

fatal: destination path 'dataframe' already exists and is not an empty directory.


In [11]:
# Código completo para la Task 3.1, sin depender de seaborn
import pandas as pd
import matplotlib.pyplot as plt
import ast

# Cargar el dataset
df = pd.read_csv('/content/dataframe/2024/dataset_Modul0_2024.csv')

# ---
# Paso 1: Crear una columna 'skill_list' de forma segura y consistente.
# ---
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return []  # En caso de error, devolvemos una lista vacía

# Parsear y normalizar las habilidades
df['skill_list'] = df['Skills'].apply(safe_literal_eval)
df['skill_list'] = df['skill_list'].apply(lambda lst: [s.strip().lower() for s in lst])

# ---
# Paso 2: Explotar la lista de habilidades y quedarnos con las top 50.
# ---
skills_exploded = df.explode('skill_list')
top_skills = skills_exploded['skill_list'].value_counts().head(50).index

# Filtramos el dataframe explotado para solo las top 50 tecnologías.
df_top50 = skills_exploded[skills_exploded['skill_list'].isin(top_skills)]

# ---
# Paso 3: Calcular la media y la mediana de candidatos para cada tecnología.
# ---
df_stats = df_top50.groupby('skill_list').agg(
    Count=('skill_list', 'size'),
    Cand_mean=('Candidates', 'mean'),
    Cand_median=('Candidates', 'median')
).reset_index()

# Redondeamos para facilitar la lectura.
df_stats['Cand_mean'] = df_stats['Cand_mean'].round(1)
df_stats['Cand_median'] = df_stats['Cand_median'].round(1)

# ---
# Paso 4: Responder a la pregunta de la Task 3.1.
# Comparamos la media y la mediana para las tecnologías dadas.
# Una tecnología "mejora su posición" si su mediana es menor que su media.
# ---
tecnologias_pregunta = ['linux', 'python', 'sql', 'java']

print("Análisis para la Task 3.1:")
print("Una tecnología mejora su posición si su mediana de candidatos es menor que su media.\n")

mejoran = []
for tech in tecnologias_pregunta:
    if tech in df_stats['skill_list'].values:
        row = df_stats[df_stats['skill_list'] == tech].iloc[0]
        mejora = row['Cand_median'] < row['Cand_mean']
        if mejora:
            mejoran.append(tech.capitalize())

        print(f"- {tech.capitalize()}:")
        print(f"    Media de candidatos: {row['Cand_mean']}")
        print(f"    Mediana de candidatos: {row['Cand_median']}")
        print(f"    ¿Mejora con la mediana en términos absolutos? {'Sí' if mejora else 'No'}")
        print()
    else:
        print(f"- {tech.capitalize()}: No está en el top 50 de tecnologías.\n")


Análisis para la Task 3.1:
Una tecnología mejora su posición si su mediana de candidatos es menor que su media.

- Linux:
    Media de candidatos: 43.5
    Mediana de candidatos: 17.0
    ¿Mejora con la mediana en términos absolutos? Sí

- Python:
    Media de candidatos: 36.4
    Mediana de candidatos: 14.0
    ¿Mejora con la mediana en términos absolutos? Sí

- Sql:
    Media de candidatos: 29.4
    Mediana de candidatos: 15.0
    ¿Mejora con la mediana en términos absolutos? Sí

- Java:
    Media de candidatos: 52.8
    Mediana de candidatos: 16.0
    ¿Mejora con la mediana en términos absolutos? Sí



La pregunta que nos haces nos piden identificar las tecnologías que mejoran en términos relativos, justo cuando dicen "con respecto al resto".

Por lo tanto, tenemos que crear un RANKING.

In [12]:
import pandas as pd
import ast

# Cargar datos
df = pd.read_csv('/content/dataframe/2024/dataset_Modul0_2024.csv')

# Parsear y normalizar habilidades
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except:
        return []

df['skill_list'] = df['Skills'].apply(safe_literal_eval)
df['skill_list'] = df['skill_list'].apply(lambda lst: [s.strip().lower() for s in lst])

# Explotar y tomar top 50
skills_exploded = df.explode('skill_list')
top_skills = skills_exploded['skill_list'].value_counts().head(50).index
df_top50 = skills_exploded[skills_exploded['skill_list'].isin(top_skills)]

# Agregar estadísticas
df_stats = df_top50.groupby('skill_list').agg(
    Count=('skill_list', 'size'),
    Cand_mean=('Candidates', 'mean'),
    Cand_median=('Candidates', 'median')
).reset_index()

# Crear rankings (1 = mejor, valor más bajo de candidatos)
df_stats = df_stats.sort_values('Cand_mean').reset_index(drop=True)
df_stats['rank_mean'] = df_stats.index + 1

df_stats = df_stats.sort_values('Cand_median').reset_index(drop=True)
df_stats['rank_median'] = df_stats.index + 1

# Ahora, para cada tecnología, ver si mejora su ranking
tecnologias = ['linux', 'python', 'sql', 'java']
print("Task 3.1: ¿Mejora la posición relativa al usar mediana en vez de media?\n")

mejoran = []
for tech in tecnologias:
    if tech in df_stats['skill_list'].values:
        row = df_stats[df_stats['skill_list'] == tech].iloc[0]
        rank_mean = row['rank_mean']
        rank_median = row['rank_median']
        mejora = rank_median < rank_mean  # sube en el ranking (número más bajo)
        if mejora:
            mejoran.append(tech.capitalize())

        print(f"- {tech.capitalize()}:")
        print(f"    Ranking con media: {rank_mean}")
        print(f"    Ranking con mediana: {rank_median}")
        print(f"    ¿Mejora posición relativa? {'Sí' if mejora else 'No'}")
        print()
    else:
        print(f"- {tech.capitalize()}: No está en el top 50.\n")

print("=== RESPUESTA FINAL ===")
print("Tecnologías que mejoran su posición *con respecto al resto*:")
for t in mejoran:
    print(f"☑️  {t}")

Task 3.1: ¿Mejora la posición relativa al usar mediana en vez de media?

- Linux:
    Ranking con media: 35
    Ranking con mediana: 34
    ¿Mejora posición relativa? Sí

- Python:
    Ranking con media: 26
    Ranking con mediana: 21
    ¿Mejora posición relativa? Sí

- Sql:
    Ranking con media: 21
    Ranking con mediana: 23
    ¿Mejora posición relativa? No

- Java:
    Ranking con media: 39
    Ranking con mediana: 27
    ¿Mejora posición relativa? Sí

=== RESPUESTA FINAL ===
Tecnologías que mejoran su posición *con respecto al resto*:
☑️  Linux
☑️  Python
☑️  Java


la interpretación correcta de la pregunta es la relativa (ranking), no la absoluta (media vs mediana).

Vamos a desglosar por qué **SQL no mejora su posición *con respecto al resto***, a pesar de que su mediana sea mucho menor que su media.

---

### 📊 Tus resultados para SQL:

- **Media**: 29.4  
- **Mediana**: 15.0  
- **Ranking con media**: 21.º  
- **Ranking con mediana**: 23.º  

✅ **Sí, su mediana < media** → esto significa que **en términos absolutos**, usar la mediana hace que SQL parezca **menos saturado** (solo 15 candidatos típicos en lugar de 29.4 en promedio).  
❌ **Pero su ranking empeora**: pasa de la **posición 21** a la **23**.

---

### ❓ ¿Cómo es posible que empeore el ranking si la mediana es más baja?

Porque **el ranking depende de cómo se mueven *todas las demás tecnologías***.

Cuando cambias de **media → mediana**, **no solo cambia el valor de SQL**, sino **el de las otras 49 tecnologías del top 50**.

#### Escenario probable:
- Tecnologías como **Python** y **Java** tienen **outliers mucho más extremos** que SQL.
  - Ej: Python tiene algunas ofertas con **miles** de candidatos, lo que infla su media a, digamos, 100, pero su mediana es 20.
  - Al pasar a mediana, **Python "cae" mucho más en valor absoluto** que SQL (de 100 → 20, una caída de 80 puntos).
- Mientras tanto, SQL solo cae de 29.4 → 15.0 (caída de ~14 puntos).
- Pero **otras tecnologías** (quizá más especializadas o con distribuciones más simétricas) **cambian poco**.
- Entonces, al reordenar **todas juntas**, **algunas tecnologías que antes estaban por debajo de SQL (peor ranking) ahora tienen valores de mediana aún más bajos que los 15.0 de SQL**, y **la adelantan**.

#### Ejemplo ilustrativo:
Supón que antes (con media):
- Tecnología X: media = 30 → **posición 22**
- **SQL**: media = 29.4 → **posición 21**
- Tecnología Y: media = 29 → **posición 20**

Después (con mediana):
- Tecnología X: mediana = **14** → ahora está **por delante** de SQL
- **SQL**: mediana = 15 → **posición 23**
- Tecnología Y: mediana = **14.5** → también por delante

→ Aunque SQL mejoró en valor absoluto (29.4 → 15), **mejoraron *más* otras tecnologías**, así que **SQL pierde posiciones relativas**.

---

### ✅ Conclusión:

- **"Mejora con la mediana" (absoluto)**: solo mira si `mediana < media` para esa tecnología.  
  → **SQL sí mejora** en este sentido.

- **"Mejora su posición con respecto al resto" (relativo)**: mira si **sube en el ranking comparado con las demás**.  
  → **SQL no mejora**, porque **otras tecnologías se benefician *más*** del cambio a mediana (especialmente Python y Java, que suben mucho: Python de 26 → 21, Java de 39 → 27).

---

### 🎯 Por tanto, la respuesta correcta a la **Task 3.1** es:

> **☑️ Linux**  
> **☑️ Python**  
> **☐ SQL**  
> **☑️ Java**